In [1]:
!pip install datasets[cli] pandas pyarrow  # pyarrow pour Parquet
# ou si tu préfères conda/mamba :
# conda install -c huggingface -c conda-forge datasets

In [2]:
# Crée un script download_dataset.py
from datasets import load_dataset
import os

output_dir = "/home/isabelle/projects/bootcamp/jedha/projet_mlops_support"  # adapte le chemin
os.makedirs(output_dir, exist_ok=True)

# Charge le dataset (télécharge auto si pas en cache)
ds = load_dataset("Tobi-Bueck/customer-support-tickets", split="train")

# Sauvegarde en Parquet (plus rapide/léger pour ton pipeline)
ds.to_parquet(os.path.join(output_dir, "train.parquet"))

# Ou en CSV si tu préfères
# ds.to_csv(os.path.join(output_dir, "train.csv"))

print(f"Dataset sauvegardé dans {output_dir}")
print(ds.column_names)  # Pour vérifier : ['queue', 'priority', 'language', 'subject', 'body', 'answer', ...]

README.md: 0.00B [00:00, ?B/s]

aa_dataset-tickets-multi-lang-5-2-50-ver(…):   0%|          | 0.00/26.0M [00:00<?, ?B/s]

(…)set-tickets-german_normalized_50_5_2.csv: 0.00B [00:00, ?B/s]

dataset-tickets-multi-lang-4-20k.csv:   0%|          | 0.00/18.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/61765 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset sauvegardé dans /home/isabelle/projects/bootcamp/jedha/projet_mlops_support
['subject', 'body', 'answer', 'type', 'queue', 'priority', 'language', 'version', 'tag_1', 'tag_2', 'tag_3', 'tag_4', 'tag_5', 'tag_6', 'tag_7', 'tag_8']


In [3]:
import pandas as pd

# Essaie d'abord le chemin relatif (si tu es dans le bon dossier)
df = pd.read_parquet('train.parquet')

# Si ça échoue avec FileNotFound, utilise le chemin absolu (adapte selon ton pwd)
# Exemples :
# df = pd.read_parquet('/home/user/PROJECTS/bootcamp/jedha/projet_mlosps_support/train.parquet')
# df = pd.read_parquet('./train.parquet')   # le ./ force relatif

# Une fois chargé :
print("Nombre de lignes et colonnes :", df.shape)
print("\nNoms des colonnes :", df.columns.tolist())

print("\nTop 15 queues (sujets/départements) :")
print(df['queue'].value_counts().head(15))

print("\nPriorities (niveaux d'urgence) :")
print(df['priority'].value_counts())

print("\nLangues :")
print(df['language'].value_counts())

print("\nExemple de ticket complet (ligne 0) :")
print(df[['subject', 'body', 'answer', 'queue', 'priority']].iloc[0].to_string())

print("\nExemple de réponse agent (première non vide) :")
print(df[df['answer'].notna()]['answer'].iloc[0][:600])

Nombre de lignes et colonnes : (61765, 16)

Noms des colonnes : ['subject', 'body', 'answer', 'type', 'queue', 'priority', 'language', 'version', 'tag_1', 'tag_2', 'tag_3', 'tag_4', 'tag_5', 'tag_6', 'tag_7', 'tag_8']

Top 15 queues (sujets/départements) :
queue
Technical Support                      14186
Product Support                         8960
Customer Service                        7420
IT Support                              5725
Billing and Payments                    4874
Returns and Exchanges                   2438
Service Outages and Maintenance         1912
Sales and Pre-Sales                     1490
Human Resources                          914
General Inquiry                          668
Pets & Animals/Pet Services              386
News                                     383
IT & Technology/Security Operations      365
Autos & Vehicles/Sales                   364
Health/Medical Services                  362
Name: count, dtype: int64

Priorities (niveaux d'urgence) :
pr

In [4]:
# Filtre recommandé
df_tech_en = df[
    (df['language'] == 'en') &
    df['queue'].isin([
        'Technical Support', 'IT Support', 'Product Support',
        'Service Outages and Maintenance', 'IT & Technology/Security Operations'
    ])
].copy()

print("Tickets après filtre :", df_tech_en.shape[0])  # ~20-25k espéré

Tickets après filtre : 17893


In [5]:
# Assure-toi que df est déjà chargé (train.parquet)
# Si pas le cas : df = pd.read_parquet('train.parquet')

df_tech_en = df[
    (df['language'] == 'en') &
    df['queue'].isin([
        'Technical Support', 'IT Support', 'Product Support',
        'Service Outages and Maintenance', 'IT & Technology/Security Operations'
    ])
].copy()

print("Nombre de tickets après filtre EN + tech :", df_tech_en.shape[0])

# Vérification rapide des queues restantes (devrait être limité aux 5 choisies)
print("\nDistribution des queues après filtre :\n", df_tech_en['queue'].value_counts())

# Répartition des urgences sur ce sous-ensemble
print("\nRépartition des priorities sur le sous-ensemble :\n", df_tech_en['priority'].value_counts())

# Répartition des urgences mappées en 3 niveaux (ajoute cette colonne)
df_tech_en['urgency_level'] = df_tech_en['priority'].apply(
    lambda p: 'high' if p in ['critical', 'high'] else 'medium' if p == 'medium' else 'low'
)

print("\nUrgence en 3 niveaux :\n", df_tech_en['urgency_level'].value_counts())

Nombre de tickets après filtre EN + tech : 17893

Distribution des queues après filtre :
 queue
Technical Support                  8149
Product Support                    5305
IT Support                         3333
Service Outages and Maintenance    1106
Name: count, dtype: int64

Répartition des priorities sur le sous-ensemble :
 priority
high      8693
medium    6767
low       2433
Name: count, dtype: int64

Urgence en 3 niveaux :
 urgency_level
high      8693
medium    6767
low       2433
Name: count, dtype: int64


In [ ]:
# Sauvegarde le sous-ensemble filtré (pour ton pipeline Airflow/Postgres) :
df_tech_en.to_parquet('train_tech_en.parquet', index=False)
print("Sauvegardé : train_tech_en.parquet")

Sauvegardé : train_tech_en.parquet


In [ ]:
# Ajoute la colonne urgence (déjà dans ton code) et vérifie distribution :
print(df_tech_en['urgency_level'].value_counts(normalize=True) * 100)  # en %

urgency_level
high      48.583245
medium    37.819259
low       13.597496
Name: proportion, dtype: float64


In [9]:
!pip install scikit-learn

  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 120.9 MB/s  0:00:00
Using cached joblib-1.5.3-py3-none-any.whl (309 kB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scikit-learn] [scikit-learn]


In [10]:
# Prépare un petit split train/val/test (pour éviter leakage RAG/SFT) :
from sklearn.model_selection import train_test_split

train_val, test = train_test_split(df_tech_en, test_size=0.15, random_state=42, stratify=df_tech_en['urgency_level'])
train, val = train_test_split(train_val, test_size=0.15, random_state=42, stratify=train_val['urgency_level'])

print(f"Train: {len(train)} | Val: {len(val)} | Test: {len(test)}")

Train: 12927 | Val: 2282 | Test: 2684


In [11]:
# Génère un exemple de document RAG (à scaler plus tard) :

def ticket_to_rag_doc(row):
    return {
        "id": str(row.name),
        "title": f"{row['queue']} - {row['subject'][:80]}",
        "content": f"""
Problème :
{row['subject']}
{row['body'][:800]}

Réponse agent typique :
{row['answer'][:600]}...
""",
        "metadata": {
            "queue": row['queue'],
            "urgency": row['urgency_level'],
            "lang": row['language']
        }
    }

# Exemple pour la première ligne
example_doc = ticket_to_rag_doc(train.iloc[0])
print(example_doc)

{'id': '54466', 'title': 'Technical Support - Inquiry about Security Protocols for Safeguarding Medical Data', 'content': '\nProblème :\nInquiry about Security Protocols for Safeguarding Medical Data\nCould you please provide more information on the security measures implemented to protect sensitive medical data from unauthorized access, breaches, and security threats in our products and services? Specifically, I am interested in detailed information on the encryption methods used, access controls, and other security features that ensure the confidentiality, integrity, and availability of medical data. I would appreciate it if you could also provide a detailed explanation of these measures. I would prefer to follow up with a call at <tel_num> to discuss this further.\n\nRéponse agent typique :\nWe will provide a detailed response regarding our medical data security protocols, including the encryption methods we use and our compliance with relevant regulations such as HIPAA. A follow-up

# Statut actuel du dataset filtré (`train_tech_en.parquet`)

Parfait, tout est aligné maintenant ! Voici un récap rapide de ton dataset filtré et prêt pour le pipeline MLOps :

## Taille
**17 893 tickets** (anglais + queues purement techniques)

## Classes sujet (queue)
4 classes naturelles et équilibrées :

| Queue | Nombre | Pourcentage |
|-------|--------|-------------|
| Technical Support | ~8 149 | ~45.5% |
| Product Support | ~5 305 | ~29.6% |
| IT Support | ~3 333 | ~18.6% |
| Service Outages and Maintenance | ~1 106 | ~6.2% |

## Urgence (urgency_level)
Bien répartie en 3 niveaux :

| Niveau | Pourcentage | Nombre |
|--------|-------------|--------|
| high | 48.6% | ~8 693 |
| medium | 37.8% | ~6 767 |
| low | 13.6% | ~2 433 |

## Split (stratifié sur urgency_level)

| Split | Nombre | Pourcentage |
|-------|--------|-------------|
| Train | 12 927 | 72% |
| Val | 2 282 | 13% |
| Test | 2 684 | 15% |

---

## Exemple de doc RAG généré (excellent)

Ton exemple pour l'ID 54466 est typique et prêt à l'emploi :

- **title** : `Technical Support - Inquiry about Security Protocols for Safeguarding Medical Data`
- **content** : combine parfaitement le problème (subject + body) + réponse agent (answer)
- **metadata** : queue, urgency, lang → super pour filtrage lors des retrieves

---

## C'est exactement ce qu'il faut pour :

- **RAG** : vectoriser ces docs (sur train uniquement)
- **Classification** : target = queue (4 classes) + urgency_level (3 classes)
- **SFT** : prompt = subject + body → output = answer

---

## Recommandation pour la suite (prochaines briques MLOps)

In [3]:
!pip install psycopg2-binary

In [5]:
import sys
!{sys.executable} -m pip install psycopg2-binary


In [6]:
import psycopg2

Note le port 5433 au lieu de 5432 !

In [9]:
# Ingestion dans Postgres (DAG Airflow 1 – ingestion & prep)
#Crée une table tickets_tech_en avec colonnes :
 # Connexion à Postgres
import psycopg2                                                                                                                                       
                                                                                                                                                        
  # Connexion à la nouvelle DB bootcamp                                                                                                                 
conn = psycopg2.connect(                                                                                                                              
    host="localhost",                                                                                                                                 
    port=5433,  # Nouveau port !                                                                                                                      
    database="support_tech",                                                                                                                          
    user="bootcamp_user",                                                                                                                             
    password="bootcamp_password"                                                                                                                      
)                                                                                                                                                     
                                                                                                                                                    
# Création de la table                                                                                                                                
cur = conn.cursor()                                                                                                                                   
cur.execute("""                                                                                                                                       
CREATE TABLE IF NOT EXISTS tickets_tech_en (                                                                                                          
    id SERIAL PRIMARY KEY,                                                                                                                            
    subject TEXT,                                                                                                                                     
    body TEXT,                                                                                                                                        
    answer TEXT,                                                                                                                                      
    queue VARCHAR(100),
    priority VARCHAR(50),
    urgency_level VARCHAR(20),
    language VARCHAR(10),
    type VARCHAR(50),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
""")
conn.commit()
print("Table tickets_tech_en créée !")

# Vérification
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
print("Tables dans support_tech :", cur.fetchall())

cur.close()
conn.close()

Table tickets_tech_en créée !
Tables dans support_tech : [('tickets_tech_en',)]


## 🚀 Prochaine Étape : Ingestion dans Postgres via Airflow

**Situation actuelle :**
* ✅ Dataset filtré et sauvegardé : `train_tech_en.parquet`
* ✅ Structure de table Postgres : Prête.

**Objectif :** Passer à l'ingestion réelle via un DAG Airflow.

---

### 1. Installation des paquets sur le VPS 📦

C'est la première étape concrète. Exécute cette commande dans ton terminal (assure-toi d'être dans ton environnement virtuel `venv` ou `conda`) :

```bash
pip install apache-airflow sqlalchemy psycopg2-binary pandas pyarrow

In [10]:
import psycopg2                                                                                                                                       
                                                                                                                                                    
conn = psycopg2.connect(                                                                                                                              
    host="localhost",                                                                                                                                 
    port=5433,                                                                                                                                        
    database="support_tech",                                                                                                                          
    user="bootcamp_user",                                                                                                                             
    password="bootcamp_password"                                                                                                                      
)                                                                                                                                                     
                                                                                                                                                    
cur = conn.cursor()                                                                                                                                   
cur.execute("""                                                                                                                                       
    SELECT column_name, data_type                                                                                                                     
    FROM information_schema.columns                                                                                                                   
    WHERE table_name = 'tickets_tech_en'                                                                                                              
    ORDER BY ordinal_position;                                                                                                                        
""")                                                                                                                                                  
print("Colonnes actuelles :")                                                                                                                         
for row in cur.fetchall():                                                                                                                            
    print(f"  - {row[0]}: {row[1]}")                                                                                                                  
                                                                                                                                                    
cur.close()                                                                                                                                           
conn.close()

Colonnes actuelles :
  - id: integer
  - subject: text
  - body: text
  - answer: text
  - queue: character varying
  - priority: character varying
  - urgency_level: character varying
  - language: character varying
  - type: character varying
  - created_at: timestamp without time zone


In [11]:
import psycopg2                                                                                                                                       
                                                                                                                                                    
conn = psycopg2.connect(
    host="localhost",
    port=5433,
    database="support_tech",
    user="bootcamp_user",
    password="bootcamp_password"
)

cur = conn.cursor()
cur.execute("""
    ALTER TABLE tickets_tech_en 
    ADD COLUMN IF NOT EXISTS tag_1 VARCHAR(100),
    ADD COLUMN IF NOT EXISTS tag_2 VARCHAR(100),
    ADD COLUMN IF NOT EXISTS tag_3 VARCHAR(100);
""")
conn.commit()
print("Colonnes tag_1, tag_2, tag_3 ajoutées !")

cur.close()
conn.close()

Colonnes tag_1, tag_2, tag_3 ajoutées !


In [12]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port=5433,
    database="support_tech",
    user="bootcamp_user",
    password="bootcamp_password"
)

cur = conn.cursor()
cur.execute("""
    SELECT column_name, data_type 
    FROM information_schema.columns 
    WHERE table_name = 'tickets_tech_en'
    ORDER BY ordinal_position;
""")

print("Structure de tickets_tech_en :")
for col in cur.fetchall():
    print(f"  {col[0]}: {col[1]}")

cur.close()
conn.close()

Structure de tickets_tech_en :
  id: integer
  subject: text
  body: text
  answer: text
  queue: character varying
  priority: character varying
  urgency_level: character varying
  language: character varying
  type: character varying
  created_at: timestamp without time zone
  tag_1: character varying
  tag_2: character varying
  tag_3: character varying


In [ ]:
# apres le trigger du dag d'ingestion on va verifier si tout est tok 

conn = psycopg2.connect(
    host="localhost",
    port=5433,
    database="support_tech",
    user="bootcamp_user",
    password="bootcamp_password"
)

cur = conn.cursor()

# Compte
cur.execute("SELECT COUNT(*) FROM tickets_tech_en;")
print(f"Total : {cur.fetchone()[0]} tickets")

# Aperçu 5 lignes
cur.execute("SELECT subject, queue, urgency_level FROM tickets_tech_en LIMIT 5;")
print("\nAperçu :")
for row in cur.fetchall():
    print(f"  - {row[0][:50]}... | {row[1]} | {row[2]}")

cur.close()
conn.close()

Total : 17893 tickets

Aperçu :
  - Account Disruption... | Technical Support | high
  - Feature Query... | Technical Support | high
  - System Interruptions... | Service Outages and Maintenance | high
  - Connectivity Problems with Printer on MacBook Pro... | Technical Support | medium
  - VPN Access Issue... | Product Support | medium


## ✅ Validation Ingestion & Plan DAG 2

### 1. Statut de l'ingestion : Succès ! 🎉

* **Total :** **17 893 tickets** dans la table `tickets_tech_en`.
* **Intégrité :** Cela correspond exactement à la taille du fichier filtré `train_tech_en.parquet` (aucune perte de lignes).
* **Qualité :** Les 5 exemples affichés montrent que les colonnes principales (`subject`, `queue`, `urgency_level`) sont bien remplies et cohérentes.

👉 **Conclusion :** Tout est OK côté ingestion, on passe à la suite !

---

### 2. Objectif du DAG 2 : Préparation & Feature Engineering 🛠️

Ce DAG va transformer les données brutes en données prêtes pour le Machine Learning.

**Les étapes clés :**

1.  **Lecture :** Charger les données depuis Postgres (`tickets_tech_en`).
2.  **Nettoyage (Cleaning) :**
    * Nettoyer `body` et `answer` (suppression balises HTML, normalisation espaces).
    * (Optionnel) Anonymisation basique (suppression téléphones/emails).
3.  **Feature Engineering :**
    * 📏 **Longueurs :** Calculer `body_length` et `answer_length`.
    * 🔑 **Keywords :** Détecter la présence de mots-clés pour raffiner les sujets (ex: *vpn, printer, network, security, login*).
    * 📊 **Ratios :** Calculer le ratio réponse/problème (pour identifier les tickets bien résolus).
4.  **Sauvegarde :** Stocker le résultat dans une nouvelle table : `tickets_tech_en_enriched`.
5.  **Monitoring (Optionnel) :** Générer un rapport **Evidently** (Qualité de données + Distribution des features).

In [15]:
# le dag 2 vient de se terminer avec succès ! 

import psycopg2

conn = psycopg2.connect(
    host="localhost", port=5433,
    database="support_tech",
    user="bootcamp_user", password="bootcamp_password"
)

cur = conn.cursor()
cur.execute("SELECT COUNT(*) FROM tickets_tech_en_enriched;")
print(f"Total : {cur.fetchone()[0]} tickets enrichis")

cur.execute("""
    SELECT body_length, answer_length, response_ratio, has_network, has_security 
    FROM tickets_tech_en_enriched LIMIT 5;
""")
for row in cur.fetchall():
    print(row)

cur.close()
conn.close()

Total : 17893 tickets enrichis
(544, 495, 0.908256880733945, 0, 1)
(646, 152, 0.23493044822256567, 0, 0)
(568, 541, 0.9507908611599297, 1, 0)
(513, 499, 0.9708171206225681, 0, 0)
(142, 39, 0.2727272727272727, 0, 1)


Explication du rapport Drift                                                                
                                                                                              
  Qu'est-ce que le Data Drift ?                                                               
                                                                                              
  Le drift c'est quand la distribution des données change entre deux périodes. On compare ici 
la première moitié des tickets (9000) vs la seconde moitié (8893).                            

  Résultat global

  Dataset Drift is NOT detected (seuil 0.5)
  → Globalement, tes données sont stables ! Pas d'alerte majeure.

  Détail par colonne

  | Colonne        | Drift ? | Distance | Signification                                     |
  |----------------|---------|----------|---------------------------------------------------|
  | body_length    | ⚠️  Oui  | 0.20     | Les tickets récents sont un peu plus longs/courts |  | answer_length  | ⚠️  Oui  | 0.14     | Les réponses varient légèrement                   |  | response_ratio | ⚠️  Oui  | 0.11     | Léger changement dans le ratio réponse/problème   |  | has_network    | ✅ Non  | 0.03     | Stable                                            |
  | has_hardware   | ✅ Non  | 0.02     | Stable                                            |
  | has_printer    | ✅ Non  | 0.02     | Stable                                            |
  | has_software   | ✅ Non  | 0.01     | Stable                                            |
  | has_security   | ✅ Non  | 0.01     | Stable                                            |

  Ce que ça veut dire pour ton projet

  1. Les catégories de tickets sont stables → Bon pour la classification !
  2. La longueur des textes varie un peu → Normal, certains tickets sont plus détaillés
  3. Pas besoin de réentraîner le modèle si ces métriques restent dans ces valeurs

  ---
  En production, tu surveillerais ce rapport régulièrement. Si le drift dépasse 0.5 → alerte pour réentraîner le modèle.


In [16]:

conn = psycopg2.connect(
    host="localhost", port=5433,
    database="support_tech",
    user="bootcamp_user", password="bootcamp_password"
)

cur = conn.cursor()

# Liste toutes les colonnes
cur.execute("""
    SELECT column_name FROM information_schema.columns 
    WHERE table_name = 'tickets_tech_en_enriched'
    ORDER BY ordinal_position;
""")
print("Colonnes de tickets_tech_en_enriched :")
for col in cur.fetchall():
    print(f"  - {col[0]}")

# Vérifie les nouvelles colonnes
cur.execute("""
    SELECT body_clean, answer_clean, body_length, answer_length, response_ratio,
            has_network, has_printer, has_security, has_hardware, has_software
    FROM tickets_tech_en_enriched LIMIT 2;
""")
print("\nExemple de données enrichies :")
for row in cur.fetchall():
    print(row)

cur.close()
conn.close()

Colonnes de tickets_tech_en_enriched :
  - subject
  - body
  - answer
  - type
  - queue
  - priority
  - language
  - version
  - tag_1
  - tag_2
  - tag_3
  - tag_4
  - tag_5
  - tag_6
  - tag_7
  - tag_8
  - urgency_level
  - body_clean
  - answer_clean
  - body_length
  - answer_length
  - response_ratio
  - has_network
  - has_printer
  - has_security
  - has_hardware
  - has_software

Exemple de données enrichies :
('Dear Customer Support Team,\\n\\nI am writing to report a significant problem with the centralized account management portal, which currently appears to be offline. This outage is blocking access to account settings, leading to substantial inconvenience. I have attempted to log in multiple times using different browsers and devices, but the issue persists.\\n\\nCould you please provide an update on the outage status and an estimated time for resolution? Also, are there any alternative ways to access and manage my account during this downtime?', 'Thank you for reachi

## ✅ Vérification et Analyse : Ingestion & Feature Engineering

**État général :** Tout est nickel ! 🎉
* **Ingestion :** Succès. La table `tickets_tech_en` contient bien **17 893 lignes**.
* **Feature Engineering :** Le DAG a fonctionné, la table `tickets_tech_en_enriched` est créée avec les colonnes nettoyées et les features calculées.

---

### 1. Analyse des Colonnes Enrichies 🧐
Les nouvelles features sont bien présentes en base :
* **Texte nettoyé :** `body_clean`, `answer_clean`
* **Métriques de longueur :** `body_length`, `answer_length`, `response_ratio`
* **Keywords (Flags) :** `has_network`, `has_printer`, `has_security`, `has_hardware`, `has_software`

---

### 2. Exemples de Données & Interprétation 📊

Voici une analyse de quelques lignes extraites (format : *Body Len / Answer Len / Ratio / Network / Printer*) :

| Ticket | Body Len | Ans Len | Ratio | Flags actifs | Interprétation |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **#1** | 544 | 495 | **0.91** | - | **Réponse détaillée :** La réponse est presque aussi longue que le problème. |
| **#2** | 646 | 152 | **0.23** | - | **Réponse concise :** Problème long mais solution courte/rapide. |
| **#3** | 568 | 541 | **0.95** | `has_network=1` | **Ticket Réseau :** Problème de connectivité avec une résolution complète. |
| **#4** | 513 | 499 | **0.97** | - | Ratio équilibré. |
| **#5** | 142 | 39 | **0.27** | `has_printer=1` | **Ticket Imprimante :** Question courte, réponse expéditive (ex: "Reboot"). |

---

### 3. Rapport de Drift (Evidently) 📉
*Comparaison : Première moitié vs Seconde moitié du dataset.*

* **🟢 Dataset Drift : NOT DETECTED**
    * *Conclusion :* Les données sont globalement stables. C'est excellent pour entraîner un modèle fiable.
* **Observations :**
    * **Léger drift sur les longueurs** (*body_length*, etc.) : Normal dans le support, les tickets tendent à devenir plus détaillés ou changer de format avec le temps.
    * **Stabilité des Keywords** (*has_**): Pas de nouveaux types de problèmes majeurs détectés.

👉 **Action :** Pas besoin de réentraîner pour l'instant. En production, ce rapport devra être généré mensuellement.

---

### 🚀 Next Steps
Priorité maintenant : **Explorer la table enrichie via SQL** pour valider la distribution des nouvelles classes.

In [17]:
conn = psycopg2.connect(                                                                    
    host="localhost", port=5433,                                                            
    database="support_tech",                                                                
    user="bootcamp_user", password="bootcamp_password"
)

cur = conn.cursor()

# Moyennes des features
cur.execute("""
    SELECT 
        AVG(body_length) AS avg_body_len,
        AVG(answer_length) AS avg_answer_len, 
        AVG(response_ratio) AS avg_ratio,
        AVG(has_network) AS pct_network,
        AVG(has_security) AS pct_security,
        AVG(has_software) AS pct_software,
        AVG(has_hardware) AS pct_hardware,
        AVG(has_printer) AS pct_printer
    FROM tickets_tech_en_enriched;
""")
row = cur.fetchone()
print("=== Moyennes des features ===")
print(f"Body length moyen     : {row[0]:.0f} caractères")
print(f"Answer length moyen   : {row[1]:.0f} caractères")
print(f"Response ratio moyen  : {row[2]:.2f}")
print(f"\n=== % de tickets par thème ===")
print(f"Network  : {row[3]*100:.1f}%")
print(f"Security : {row[4]*100:.1f}%")
print(f"Software : {row[5]*100:.1f}%")
print(f"Hardware : {row[6]*100:.1f}%")
print(f"Printer  : {row[7]*100:.1f}%")

# Top 5 tickets sécurité
print("\n=== Top 5 tickets Security ===")
cur.execute("""
    SELECT subject, queue, urgency_level, LEFT(body_clean, 100) as body_preview
    FROM tickets_tech_en_enriched
    WHERE has_security = 1
    LIMIT 5;
""")
for row in cur.fetchall():
    print(f"\n[{row[2]}] {row[1]}")
    print(f"  Subject: {row[0]}")
    print(f"  Body: {row[3]}...")

cur.close()
conn.close()


=== Moyennes des features ===
Body length moyen     : 371 caractères
Answer length moyen   : 368 caractères
Response ratio moyen  : 1.46

=== % de tickets par thème ===
Network  : 10.9%
Security : 23.3%
Software : 36.4%
Hardware : 4.9%
Printer  : 0.8%

=== Top 5 tickets Security ===

[high] Technical Support
  Subject: Account Disruption
  Body: Dear Customer Support Team,\n\nI am writing to report a significant problem with the centralized acc...

[high] Technical Support
  Subject: Customer Support for Data Breach
  Body: A healthcare organization identified unauthorized access attempts. Password resets have been carried...

[medium] IT Support
  Subject: Immediate Help Needed: Technical Problem with Cloud SaaS Service
  Body: Dear Customer Support Team,\n\nI am submitting a report regarding a technical problem encountered wi...

[medium] Service Outages and Maintenance
  Subject: Query About Future Service Disruptions and Maintenance Timelines
  Body: Dear Customer Support Team,\n\n

### 🚀 Synthèse et Suggestions : Exploitation des Keywords

Parfait, on avance bien ! Voici une synthèse rapide de ce que tu as partagé, et des suggestions pour exploiter ces infos dans ton projet bootcamp.

#### 1. Analyse des keywords / thèmes émergents 📊
Après avoir calculé les features `has_*` dans le DAG de prep, tu as ces proportions approximatives sur les **17 893 tickets** (basé sur *AVG(has_*) * 100*) :

* **Software : 36.4%**
    * → Thème dominant : bugs, mises à jour, installations, crashes, versions de produits.
    * → Très cohérent avec *Product Support* (29.6%) et *Technical Support*.
* **Hardware : 4.9%**
    * → Moins fréquent : laptops, écrans, disques, CPU, MacBook.
    * → Souvent lié à *IT Support* (18.6%).
* **Printer : 0.8%**
    * → Rare : imprimantes, scan, print jobs.
    * → Niche mais concret pour le support technique.

---

#### 2. Proposition de raffinement des classes 🛠️
C'est une bonne base pour raffiner tes classes sujet au-delà des 4 queues principales. Voici une proposition de **6 à 8 classes** :

| Classe élargie (Proposition) | Base (Queue) + Keywords | % Approx. | Commentaire |
| :--- | :--- | :--- | :--- |
| **Software / Product** | Product Support + `has_software` | ~36–40% | Dominant, bugs & apps |
| **General Technical Support** | Technical Support (sans keywords) | ~30–35% | "Catch-all" technique |
| **IT Infrastructure / Network** | IT Support + `has_network` | ~15–20% | VPN, connexion, infra |
| **Hardware / Device** | IT Support + `has_hardware` | ~5–7% | Machines physiques |
| **Service Outages / Maintenance** | Service Outages... | ~6% | Pannes planifiées |
| **Security / Access** | `has_security` | ~2–5% | Incidents sensibles (faible % mais critique) |
| **Printer / Peripherals** | `has_printer` | ~1% | Très spécifique |

👉 **Action :** Tu peux créer une colonne `sub_category` ou `refined_queue` dans la table enrichie pour implémenter cette logique.

### 🔐 Analyse des Top 5 tickets avec `has_security = 1`

Ces 5 exemples montrent bien la pertinence du thème sécurité dans ton dataset :

* **[High] Technical Support – Account Disruption**
    * *Analyse :* Problème de compte centralisé compromis → urgence haute → **typique incident de sécurité** (accès non autorisé).
* **[High] Technical Support – Customer Support for Data Breach**
    * *Analyse :* Accès non autorisé sur organisation santé → password resets → urgence haute → **Data Breach clair**.
* **[Medium] IT Support – Immediate Help Needed: Technical Problem with Cloud SaaS Service**
    * *Analyse :* Problème technique SaaS → pas explicitement breach, mais peut inclure une vulnérabilité.
* **[Medium] Service Outages and Maintenance – Query About Future Service Disruptions**
    * *Analyse :* Question sur interruptions futures → moins "sécurité", plus maintenance → **Potentiel faux positif** du mot-clé.
* **[High] Technical Support – Assistance Required for Data Security Incident**
    * *Analyse :* Breach dû à mesures de sécurité obsolètes → urgence haute → **Cas critique santé**.

---

### 💡 Insights Clés

1.  **Corrélation Urgence :** La plupart des tickets "security" sont **High Urgency** → logique pour des incidents sensibles (fuite de données, accès piraté).
2.  **Catégories d'origine :** Souvent cachés dans *Technical Support* ou *IT Support*.
3.  **Thèmes récurrents :** `unauthorized access`, `data breach`, `outdated security`, `password reset`, `healthcare context`.

### 🚀 Suggestions pour la suite du projet

* **Améliorer le raffinement des classes :**
    * Ajoute plus de **keywords** pour la classe *Security* (ex. `breach`, `unauthorized`, `hack`, `malware`, `encrypt`, `hipaa`).
    * Ajoute plus de keywords pour *Network* (`vpn`, `wifi`, `router`, `connectivity`).
    * *Action :* Relance le DAG de préparation si besoin pour affiner la colonne `refined_queue`.

In [18]:
# on ajoute une colonne refined_queue pour créer une classification plus fine en 7 catégories
                                                                                            
conn = psycopg2.connect(                                                                    
    host="localhost", port=5433,
    database="support_tech",
    user="bootcamp_user", password="bootcamp_password"
)

cur = conn.cursor()

# Ajoute la colonne refined_queue
cur.execute("""
    ALTER TABLE tickets_tech_en_enriched 
    ADD COLUMN IF NOT EXISTS refined_queue VARCHAR(100);
""")

# Met à jour avec la logique de classification
cur.execute("""
    UPDATE tickets_tech_en_enriched
    SET refined_queue = CASE
        WHEN has_security = 1 THEN 'Security / Access'
        WHEN has_printer = 1 THEN 'Printer / Peripherals'
        WHEN has_hardware = 1 THEN 'Hardware / Device'
        WHEN has_network = 1 THEN 'Network / Infrastructure'
        WHEN has_software = 1 OR queue = 'Product Support' THEN 'Software / Product'
        WHEN queue = 'Service Outages and Maintenance' THEN 'Service Outages'
        ELSE 'General Technical Support'
    END;
""")

conn.commit()
print("Colonne refined_queue créée et mise à jour !")

# Vérifie la distribution
cur.execute("""
    SELECT refined_queue, COUNT(*) as count,  
            ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 1) as pct
    FROM tickets_tech_en_enriched
    GROUP BY refined_queue
    ORDER BY count DESC;
""")

print("\n=== Distribution des refined_queue ===")
for row in cur.fetchall():
    print(f"{row[0]:<30} : {row[1]:>5} ({row[2]}%)")

cur.close()
conn.close()

Colonne refined_queue créée et mise à jour !

=== Distribution des refined_queue ===
Software / Product             :  6604 (36.9%)
General Technical Support      :  4679 (26.1%)
Security / Access              :  4165 (23.3%)
Network / Infrastructure       :  1196 (6.7%)
Hardware / Device              :   745 (4.2%)
Service Outages                :   466 (2.6%)
Printer / Peripherals          :    38 (0.2%)


## 📊 Analyse de la distribution : `refined_queue`

Parfait ! La colonne `refined_queue` est créée et mise à jour. On a maintenant une granularité beaucoup plus fine et réaliste pour ton agent support technique interne.

### 1. Distribution finale des classes (sur 17 893 tickets)

| Classe `refined_queue` | Tickets | % | Commentaire / Utilisation projet |
| :--- | :--- | :--- | :--- |
| **Software / Product** | 6 604 | **36.9 %** | Dominant → bugs, apps, mises à jour, features |
| **General Technical Support** | 4 679 | **26.1 %** | "Catch-all" technique général |
| **Security / Access** | 4 165 | **23.3 %** | **Très élevé !** → accès, login, breach, password |
| **Network / Infrastructure** | 1 196 | **6.7 %** | VPN, connexion, wifi, router, infra |
| **Hardware / Device** | 745 | **4.2 %** | PC, MacBook, écran, disque, périphériques |
| **Service Outages** | 466 | **2.6 %** | Pannes, maintenance planifiée |
| **Printer / Peripherals** | 38 | **0.2 %** | Très rare → imprimantes, scan |

### 2. Observations Clés 🧐
* **Concentration :** 3 classes couvrent déjà **~86 %** des tickets (Software + General + Security) → très bon équilibre pour un modèle de classification.
* **Sécurité :** La classe *Security* est beaucoup plus présente (23.3 %) que ce que les queues brutes laissaient penser (0.8–5 %). Grâce aux keywords, on capture bien les incidents sensibles (*breach, unauthorized, outdated security*).
* **Optimisation :** *Printer* reste marginal. Tu peux le fusionner dans *Hardware / Device* pour réduire à 6 classes.
* **Conclusion :** 7 classes (ou 6 si merge) → pile dans la cible 6–12, et beaucoup plus représentatif d'un vrai helpdesk IT (software dominant, sécurité critique, hardware/réseau secondaires).

---

### 3. Implications pour le projet Bootcamp 🚀

#### A. Classification ML (Prochain DAG)
* **Target principal :** `refined_queue` (7 classes).
* **Target secondaire :** `urgency_level` (3 classes).
* **Features :**
    * *Text :* TF-IDF ou MiniLM embeddings sur `body_clean` + `subject`.
    * *Numériques :* `body_length`, `answer_length`, `response_ratio`, `has_*` (déjà calculés).
* **Modèle :** XGBoost (multi-class).
* **Avantage :** Les keywords (`has_*`) deviennent des features puissantes → boost de performance.

#### B. RAG (Retrieval Augmented Generation)
* Les docs RAG peuvent inclure `refined_queue` dans les **metadata** pour filtrer.
* *Exemple :* "Récupère les tickets similaires dans la catégorie *Security / Access* seulement".

#### C. Monitoring (Evidently)
* Tu peux maintenant monitorer le **Drift** sur `refined_queue` (distribution des thèmes) en plus des longueurs et keywords.
* *Exemple :* Si *Security / Access* augmente fortement → alerte "augmentation des incidents sécurité".

---

### 4. Actions Immédiates (SQL) 🛠️

**Vérifier la distribution :**
```sql
SELECT refined_queue,
       COUNT(*) AS count,
       ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 1) AS pct
FROM tickets_tech_en_enriched
GROUP BY refined_queue
ORDER BY count DESC;
```
**(Optionnel) Fusionner Printer dans Hardware :**
```sql
UPDATE tickets_tech_en_enriched
SET refined_queue = 'Hardware / Device'
WHERE refined_queue = 'Printer / Peripherals';
```

In [19]:

conn = psycopg2.connect(
    host="localhost", port=5433,
    database="support_tech",
    user="bootcamp_user", password="bootcamp_password"
)

cur = conn.cursor()

# Fusionne Printer dans Hardware
cur.execute("""
    UPDATE tickets_tech_en_enriched
    SET refined_queue = 'Hardware / Device'
    WHERE refined_queue = 'Printer / Peripherals';
""")
conn.commit()
print(f"Fusionné : {cur.rowcount} tickets Printer → Hardware")

# Nouvelle distribution
cur.execute("""
    SELECT refined_queue, COUNT(*) as count, 
            ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 1) as pct
    FROM tickets_tech_en_enriched
    GROUP BY refined_queue
    ORDER BY count DESC;
""")

print("\n=== Distribution finale (6 classes) ===")
for row in cur.fetchall():
    print(f"{row[0]:<30} : {row[1]:>5} ({row[2]}%)")

cur.close()
conn.close()

Fusionné : 38 tickets Printer → Hardware

=== Distribution finale (6 classes) ===
Software / Product             :  6604 (36.9%)
General Technical Support      :  4679 (26.1%)
Security / Access              :  4165 (23.3%)
Network / Infrastructure       :  1196 (6.7%)
Hardware / Device              :   783 (4.4%)
Service Outages                :   466 (2.6%)


In [20]:

conn = psycopg2.connect(
    host="localhost", port=5433,
    database="support_tech",
    user="bootcamp_user", password="bootcamp_password"
)

cur = conn.cursor()

cur.execute("""
    SELECT refined_queue,
            COUNT(*) AS count,
            ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 1) AS pct
    FROM tickets_tech_en_enriched
    GROUP BY refined_queue
    ORDER BY count DESC;
""")

print("=== Distribution refined_queue ===")
for row in cur.fetchall():
    print(f"{row[0]:<30} : {row[1]:>5} ({row[2]}%)")

cur.close()
conn.close()

=== Distribution refined_queue ===
Software / Product             :  6604 (36.9%)
General Technical Support      :  4679 (26.1%)
Security / Access              :  4165 (23.3%)
Network / Infrastructure       :  1196 (6.7%)
Hardware / Device              :   783 (4.4%)
Service Outages                :   466 (2.6%)


In [21]:
conn = psycopg2.connect(
    host="localhost", port=5433,
    database="support_tech",
    user="bootcamp_user", password="bootcamp_password"
)

cur = conn.cursor()

cur.execute("""
    SELECT subject, LEFT(body_clean, 150) as preview, urgency_level
    FROM tickets_tech_en_enriched
    WHERE refined_queue = 'Network / Infrastructure'
    LIMIT 5;
""")

print("=== Exemples tickets Network / Infrastructure ===\n")
for row in cur.fetchall():
    print(f"[{row[2]}] {row[0]}")
    print(f"   {row[1]}...\n")

cur.close()
conn.close()

=== Exemples tickets Network / Infrastructure ===

[high] None
   Facing connectivity problems with Google Nest Wifi Router...

[high] Service Disruption
   Encountering several device-related service interruptions impacting workflow. Attempts to resolve by rebooting devices and checking internet connectiv...

[low] Problem with investment data failing to update properly
   Description: Investment data is not updating as expected. Possible causes: API connection problems or out-of-date software. Efforts made: Restarted eq...

[high] Project Sync Failure
   Below is a brief issue description: The project sync has failed. It might be related to a MongoDB connection problem. Steps taken: Restarted Docker, v...

[medium] None
   We are encountering connectivity problems with the SaaS project management tool on my iMac, which might be related to network instability with the Goo...



### 🌐 Analyse : Focus sur "Network / Infrastructure"

On voit que ce sont tous des problèmes de connexion/réseau :

| Ticket | Problème |
| :--- | :--- |
| **Google Nest Wifi Router** | Connectivity problems |
| **Service Disruption** | Internet connectivity issues |
| **Investment data** | API connection problems |
| **Project Sync Failure** | MongoDB connection problem |
| **SaaS tool** | Network instability |

🔍 **Mots-clés détectés :** `connectivity`, `connection`, `network`, `wifi`, `router`

---

### 📋 Récapitulatif final des 6 classes

Voici la structure finale de ta colonne cible `refined_queue` :

| Classe `refined_queue` | C'est quoi ? (Description) |
| :--- | :--- |
| **Software / Product** | Bugs, apps, updates, installations |
| **General Technical Support** | Problèmes techniques généraux (Catch-all) |
| **Security / Access** | Login, passwords, data breach, accès |
| **Network / Infrastructure** | VPN, wifi, connexion, réseau |
| **Hardware / Device** | PC, laptop, écran, imprimante |
| **Service Outages** | Pannes, maintenance planifiée |